# DeepEval을 활용한 Agent 평가 가이드

이 노트북에서는 DeepEval을 사용하여 AI Agent를 체계적으로 평가하는 방법을 단계별로 설명합니다.

## 목차

## 학습 목표
- DeepEval의 핵심 기능 이해
- 체계적인 평가 메트릭 설계
- 자동화된 평가 파이프라인 구축
- 평가 결과 분석 및 개선점 도출


## 1. 환경 설정 및 라이브러리 설치

먼저 필요한 라이브러리들을 설치하고 환경을 설정합니다.


In [10]:
# 필요한 라이브러리 설치
# %pip install deepeval langgraph langchain-openai tavily-python

In [ ]:
# 라이브러리 import
import os
import json
from typing import Dict, List, Any

# DeepEval import
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

# OpenAI 및 Tavily import
from openai import OpenAI
from tavily import TavilyClient

os.environ["OPENAI_API_KEY"] = (
    "your-api-key"
)
os.environ["TAVILY_API_KEY"] = "your-api-key"

## 2. 기본 Agent 구현

간단한 Agent를 구현합니다. 이 Agent는 다음과 같은 도구들을 사용할 수 있습니다:
- 웹 검색 (Tavily API)
- 현재 시간 조회
- 간단한 수학 계산


In [12]:
class BasicAgent:
    """기본적인 Agent 구현"""

    def __init__(self):
        # OpenAI 클라이언트 초기화
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        # Tavily 클라이언트 초기화
        self.tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

        # 도구들 정의
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "search_web",
                    "description": "웹에서 정보를 검색합니다",
                    "parameters": {
                        "type": "object",
                        "properties": {"query": {"type": "string", "description": "검색할 쿼리"}},
                        "required": ["query"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_current_time",
                    "description": "현재 시간을 반환합니다",
                    "parameters": {"type": "object", "properties": {}, "required": []},
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "calculate",
                    "description": "간단한 수학 계산을 수행합니다",
                    "parameters": {
                        "type": "object",
                        "properties": {"expression": {"type": "string", "description": "계산할 수식 (예: 2 + 3 * 4)"}},
                        "required": ["expression"],
                    },
                },
            },
        ]

    def search_web(self, query: str) -> str:
        """웹 검색 도구"""
        try:
            results = self.tavily_client.search(
                query=query, max_results=3, include_domains=["wikipedia.org", "news.ycombinator.com", "github.com"]
            )

            search_results = []
            for result in results.get("results", []):
                search_results.append(
                    {
                        "title": result.get("title", ""),
                        "content": result.get("content", ""),
                        "url": result.get("url", ""),
                    }
                )

            return json.dumps(search_results, ensure_ascii=False, indent=2)
        except Exception as e:
            return f"검색 중 오류 발생: {str(e)}"

    def get_current_time(self) -> str:
        """현재 시간 도구"""
        from datetime import datetime

        now = datetime.now()
        return f"현재 시간: {now.strftime('%Y-%m-%d %H:%M:%S')}"

    def calculate(self, expression: str) -> str:
        """계산 도구"""
        try:
            # 안전한 계산을 위해 제한된 연산만 허용
            allowed_chars = set("0123456789+-*/.() ")
            if not all(c in allowed_chars for c in expression):
                return "안전하지 않은 수식입니다. 숫자와 기본 연산자만 사용하세요."

            result = eval(expression)
            return f"{expression} = {result}"
        except Exception as e:
            return f"계산 오류: {str(e)}"

    def run(self, user_input: str) -> Dict[str, Any]:
        """Agent 실행"""
        messages = [
            {
                "role": "system",
                "content": """당신은 도움이 되는 AI 어시스턴트입니다. 
사용자의 질문에 정확하고 유용한 답변을 제공하세요.

사용 가능한 도구들:
1. search_web: 웹에서 정보를 검색합니다
2. get_current_time: 현재 시간을 알려줍니다
3. calculate: 간단한 수학 계산을 수행합니다

도구를 사용할 때는 정확한 매개변수를 제공하세요.""",
            },
            {"role": "user", "content": user_input},
        ]

        tools_called = []
        max_iterations = 5
        iteration = 0

        while iteration < max_iterations:
            # OpenAI API 호출
            response = self.client.chat.completions.create(
                model="gpt-4o-mini", messages=messages, tools=self.tools, tool_choice="auto", temperature=0.1
            )

            message = response.choices[0].message
            messages.append(message)

            # 도구 호출이 있는지 확인
            if hasattr(message, "tool_calls") and message.tool_calls:
                for tool_call in message.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)

                    # 도구 호출 기록
                    tools_called.append({"tool_name": tool_name, "parameters": tool_args, "call_id": tool_call.id})

                    # 도구 실행
                    if tool_name == "search_web":
                        result = self.search_web(tool_args["query"])
                    elif tool_name == "get_current_time":
                        result = self.get_current_time()
                    elif tool_name == "calculate":
                        result = self.calculate(tool_args["expression"])
                    else:
                        result = f"알 수 없는 도구: {tool_name}"

                    # 도구 결과를 메시지에 추가
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result})

                iteration += 1
            else:
                # 도구 호출이 없으면 최종 답변
                break

        # 최종 답변 추출
        final_answer = ""
        for message in reversed(messages):
            if hasattr(message, "role") and message.role == "assistant" and not hasattr(message, "tool_calls"):
                final_answer = message.content
                break
            elif isinstance(message, dict) and message.get("role") == "assistant" and not message.get("tool_calls"):
                final_answer = message["content"]
                break

        return {"input": user_input, "output": final_answer, "tools_called": tools_called, "iterations": iteration}


print("BasicAgent 클래스 정의 완료!")

BasicAgent 클래스 정의 완료!


In [13]:
# Agent 테스트
agent = BasicAgent()

# 간단한 테스트
test_input = "현재 시간을 알려주세요"
result = agent.run(test_input)

print(f"입력: {result['input']}")
print(f"출력: {result['output']}")
print(f"도구 호출: {len(result['tools_called'])}개")
for tool_call in result["tools_called"]:
    print(f"  - {tool_call['tool_name']}: {tool_call['parameters']}")

입력: 현재 시간을 알려주세요
출력: 
도구 호출: 1개
  - get_current_time: {}


## 3. DeepEval 기본 사용법

DeepEval의 핵심 구성 요소들을 살펴보겠습니다:
- **LLMTestCase**: 개별 테스트 케이스
- **EvaluationDataset**: 테스트 케이스들의 컬렉션
- **GEval**: 커스텀 평가 메트릭


In [14]:
# LLMTestCase 생성 예시
test_case = LLMTestCase(
    input="현재 시간을 알려주세요",
    actual_output="현재 시간: 2024-01-15 14:30:25",
    expected_output="현재 시간 정보를 제공하는 응답",
    context=["사용자가 현재 시간을 요청함"],
    comments="시간 조회 기능 테스트",
)

print("LLMTestCase 생성 완료!")
print(f"입력: {test_case.input}")
print(f"실제 출력: {test_case.actual_output}")
print(f"예상 출력: {test_case.expected_output}")
print(f"컨텍스트: {test_case.context}")
print(f"설명: {test_case.comments}")

LLMTestCase 생성 완료!
입력: 현재 시간을 알려주세요
실제 출력: 현재 시간: 2024-01-15 14:30:25
예상 출력: 현재 시간 정보를 제공하는 응답
컨텍스트: ['사용자가 현재 시간을 요청함']
설명: 시간 조회 기능 테스트


In [16]:
# EvaluationDataset 생성 및 관리
dataset = EvaluationDataset()

# 여러 테스트 케이스 추가
test_cases = [
    LLMTestCase(
        input="현재 시간을 알려주세요",
        actual_output="현재 시간: 2024-01-15 14:30:25",
        expected_output="현재 시간 정보를 제공하는 응답",
        context=["사용자가 현재 시간을 요청함"],
        comments="시간 조회 기능 테스트",
    ),
    LLMTestCase(
        input="2 + 3 * 4를 계산해주세요",
        actual_output="2 + 3 * 4 = 14",
        expected_output="14",
        context=["사용자가 수학 계산을 요청함"],
        comments="계산 기능 테스트",
    ),
    LLMTestCase(
        input="Python 프로그래밍의 장점을 알려주세요",
        actual_output="Python은 간결하고 읽기 쉬운 문법을 가진 프로그래밍 언어입니다...",
        expected_output="Python의 주요 장점들을 설명하는 응답",
        context=["사용자가 Python에 대한 정보를 요청함"],
        comments="일반적인 질문 답변 테스트",
    ),
]

# 데이터셋에 테스트 케이스들 추가
for test_case in test_cases:
    dataset.add_test_case(test_case)

print(f"데이터셋에 {len(dataset.test_cases)}개 테스트 케이스 추가됨")
print(f"데이터셋 통계:")
print(f"  - 총 테스트 케이스: {len(dataset.test_cases)}")
print(f"  - 총 골든 데이터: {len(dataset.goldens)}")

데이터셋에 3개 테스트 케이스 추가됨
데이터셋 통계:
  - 총 테스트 케이스: 3
  - 총 골든 데이터: 0


## 4. 합성 데이터 생성

DeepEval의 Synthesizer를 사용하여 문서에서 자동으로 평가 데이터를 생성합니다.


In [17]:
# 샘플 문서 생성
sample_documents = {
    "ai_guide.txt": """
AI Agent는 인공지능 기반의 자율적인 소프트웨어 프로그램입니다.

Agent의 주요 특징:
1. 자율성: 인간의 개입 없이 독립적으로 작업 수행
2. 반응성: 환경 변화에 적절히 반응
3. 능동성: 목표 달성을 위해 능동적으로 행동
4. 사회성: 다른 Agent나 시스템과 상호작용

Agent의 핵심 구성 요소:
- 인식 모듈: 환경 정보 수집 및 해석
- 추론 모듈: 정보 분석 및 의사결정
- 행동 모듈: 결정된 행동 실행
- 학습 모듈: 경험을 통한 성능 개선

현대 AI Agent는 자연어 처리, 컴퓨터 비전, 강화학습 등의 기술을 활용합니다.
ChatGPT, Claude, Gemini 등의 대화형 AI도 Agent의 한 형태입니다.
""",
    "evaluation_guide.txt": """
AI Agent 평가는 Agent의 성능과 품질을 측정하는 중요한 과정입니다.

평가의 주요 목적:
1. 성능 측정: Agent가 목표를 얼마나 잘 달성하는지 측정
2. 품질 보증: 출력의 정확성과 일관성 확인
3. 개선점 식별: 약점과 개선이 필요한 영역 파악
4. 비교 분석: 다른 Agent나 버전과의 성능 비교

평가 방법론:
- 정량적 평가: 수치적 지표를 통한 객관적 측정
- 정성적 평가: 전문가 판단을 통한 주관적 평가
- 사용자 평가: 실제 사용자의 만족도와 피드백
- 자동 평가: AI 모델을 활용한 자동화된 평가

주요 평가 메트릭:
- 정확도: 올바른 답변의 비율
- 완성도: 요청사항 충족 정도
- 일관성: 동일한 입력에 대한 일관된 출력
- 효율성: 리소스 사용량과 처리 시간
""",
}

# 문서들을 파일로 저장
for filename, content in sample_documents.items():
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)

print("샘플 문서 생성 완료!")
print(f"생성된 문서: {list(sample_documents.keys())}")

샘플 문서 생성 완료!
생성된 문서: ['ai_guide.txt', 'evaluation_guide.txt']


In [20]:
# Synthesizer를 사용한 합성 데이터 생성
synthesizer = Synthesizer()

# 문서에서 골든 데이터 생성
document_files = list(sample_documents.keys())
goldens = synthesizer.generate_goldens(document_files)  # 다른 메서드일 가능성 큼

print(f"생성된 골든 데이터: {len(goldens)}개")

# 생성된 골든 데이터 출력
for i, golden in enumerate(goldens, 1):
    print(f"\n골든 데이터 {i}:")
    print(f"  입력: {golden.input}")
    print(f"  예상 출력: {golden.expected_output[:100]}...")
    print(f"  컨텍스트: {len(golden.context)}개 청크")

AttributeError: 'Synthesizer' object has no attribute 'generate_goldens'

## 5. 평가 메트릭 구현

Agent 평가를 위한 커스텀 메트릭들을 구현합니다.


In [ ]:
# 커스텀 평가 메트릭 생성
task_completion_metric = GEval(
    name="Task Completion",
    criteria="""
Agent가 사용자의 요청을 얼마나 완전하게 수행했는지 평가하세요.

평가 기준:
1. 사용자 요청의 핵심 의도를 파악했는가?
2. 요청된 작업을 적절히 수행했는가?
3. 결과가 사용자의 기대에 부합하는가?

점수는 1-10 척도로 평가하며, 10점은 완벽한 작업 완수를 의미합니다.
""",
    evaluation_params=[
        # YOUR_CODE
    ],
    evaluation_steps=[
        "사용자 요청의 핵심 요구사항을 식별하세요.",
        "Agent의 출력에서 각 요구사항의 충족 여부를 확인하세요.",
        "전체적인 작업 완성도를 종합적으로 평가하세요.",
    ],
    model="gpt-4o-mini",
)

response_quality_metric = GEval(
    name="Response Quality",
    criteria="""
Agent 응답의 품질을 평가하세요.

평가 기준:
1. 응답이 정확하고 유용한가?
2. 응답이 명확하고 이해하기 쉬운가?
3. 응답이 완전하고 포괄적인가?

점수는 1-10 척도로 평가하며, 10점은 완벽한 응답을 의미합니다.
""",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    evaluation_steps=["응답의 정확성을 평가하세요.", "응답의 명확성을 평가하세요.", "응답의 완성도를 평가하세요."],
    model="gpt-4o-mini",
)

print("커스텀 평가 메트릭 생성 완료!")
print(f"생성된 메트릭:")
print(f"  - {task_completion_metric.name}")
print(f"  - {response_quality_metric.name}")

## 6. 전체 평가 파이프라인

이제 모든 구성 요소를 연결하여 완전한 평가 파이프라인을 구축합니다.


In [ ]:
# 1단계: 골든 데이터를 테스트 케이스로 변환
print("=== 1단계: 테스트 케이스 변환 ===")
test_cases = []
for golden in goldens:
    test_case = LLMTestCase(
        # YOUR_CODE
    )
    test_cases.append(test_case)

print(f"변환된 테스트 케이스: {len(test_cases)}개")

In [ ]:
# 2단계: Agent 실행
print("\n=== 2단계: Agent 실행 ===")
for i, test_case in enumerate(test_cases):
    try:
        result = agent.run(test_case.input)
        test_case.actual_output = result["output"]

        # 도구 호출 정보를 추가 메타데이터에 저장
        test_case.additional_metadata = {"tools_called": result["tools_called"], "iterations": result["iterations"]}

        print(f"테스트 케이스 {i+1} 완료")

    except Exception as e:
        test_case.actual_output = f"오류 발생: {str(e)}"
        test_case.additional_metadata = {"error": str(e)}
        print(f"테스트 케이스 {i+1} 실패: {str(e)}")

In [ ]:
# 3단계: 메트릭 평가
print("\n=== 3단계: 메트릭 평가 ===")
metrics = [task_completion_metric, response_quality_metric]
evaluation_results = {
    "test_cases": [],
    "overall_scores": {}
}

for i, test_case in enumerate(test_cases):
    print(f"테스트 케이스 {i+1} 평가 중...")
    
    case_results = {
        "input": test_case.input,
        "actual_output": test_case.actual_output,
        "expected_output": test_case.expected_output,
        "scores": {},
        "details": {}
    }
    
    # 각 메트릭으로 평가
    for metric in metrics:
        try:
            metric_name = metric.name
            metric.# YOUR_CODE
            score = # YOUR_CODE
            reason = getattr(metric, 'reason', 'N/A')
            
            case_results["scores"][metric_name] = score
            case_results["details"][metric_name] = {
                "score": score,
                "reason": reason
            }
            
        except Exception as e:
            metric_name = metric.name
            case_results["scores"][metric_name] = 0.0
            case_results["details"][metric_name] = {
                "score": 0.0,
                "reason": f"평가 실패: {str(e)}"
            }
    
    evaluation_results["test_cases"].append(case_results)

# 전체 점수 계산
for metric in metrics:
    metric_name = metric.name
    scores = [case["scores"].get(metric_name, 0.0) for case in evaluation_results["test_cases"]]
    if scores:
        evaluation_results["overall_scores"][metric_name] = {
            "average": sum(scores) / len(scores),
            "min": min(scores),
            "max": max(scores),
            "std": (sum((x - sum(scores)/len(scores))**2 for x in scores) / len(scores))**0.5
        }

print("메트릭 평가 완료!")


## 7. 결과 분석 및 시각화

평가 결과를 분석하고 시각화합니다.


In [ ]:
# 전체 평가 결과 출력
print("=== 전체 평가 결과 ===")
for metric_name, scores in evaluation_results["overall_scores"].items():
    print(f"\n{metric_name}:")
    print(f"  평균: {scores['average']:.2f}")
    print(f"  최소: {scores['min']:.2f}")
    print(f"  최대: {scores['max']:.2f}")
    print(f"  표준편차: {scores['std']:.2f}")

# 성능 등급 분류
print(f"\n=== 성능 등급 분류 ===")
for metric_name, scores in evaluation_results["overall_scores"].items():
    avg_score = scores["average"]
    if avg_score >= 8.0:
        grade = "우수"
    elif avg_score >= 6.0:
        grade = "양호"
    elif avg_score >= 4.0:
        grade = "보통"
    else:
        grade = "개선 필요"

    print(f"  {metric_name}: {grade} ({avg_score:.2f})")

In [ ]:
# 개별 테스트 케이스 분석
print("\n=== 개별 테스트 케이스 분석 ===")
for i, case in enumerate(evaluation_results["test_cases"]):
    print(f"\n테스트 케이스 {i+1}:")
    print(f"  입력: {case['input'][:100]}...")
    print(f"  점수:")
    for metric_name, score in case["scores"].items():
        print(f"    {metric_name}: {score:.2f}")

    # 도구 호출 정보 출력
    if "tools_called" in case.get("additional_metadata", {}):
        tools = case["additional_metadata"]["tools_called"]
        print(f"  도구 호출: {len(tools)}개")
        for tool in tools:
            print(f"    - {tool['tool_name']}: {tool['parameters']}")

In [ ]:
# 결과를 JSON 파일로 저장
with open("evaluation_results.json", "w", encoding="utf-8") as f:
    json.dump(evaluation_results, f, ensure_ascii=False, indent=2, default=str)

print("\n=== 평가 완료 ===")
print("결과가 evaluation_results.json에 저장되었습니다.")

# 임시 파일 정리
import glob

for file in glob.glob("*.txt"):
    os.remove(file)
print("임시 파일들이 정리되었습니다.")